In [132]:
# Clear all current variables
import sys
#sys.modules[__name__].__dict__.clear()

import math
import time
import random
import collections
import numpy as np
from numba import jit
import matplotlib.pyplot as plt

point = collections.namedtuple('Point', ['Distance', 'Combs', 'CombC'])
point2 = collections.namedtuple('Point2', ['Time','Kinetic', 'Potential'])


In [133]:
# INIT
N = 3                       # Integer value
n = math.pow(N,3)*4         # Molecule count
n = int(n)
dens = 0.8                  # dens = N / V
V = n / dens                # Volume density
D = math.pow(V,1/3)         # Size of the box
CombC    = 0                # Count for the combinations
MinDist  = 1.8              # This corresponds with Force = 0.05

Kin     = 0.0
Pot     = 0.0
Tot     = 0.0

In [189]:
def InitPos():
    pos = np.zeros((n,3))
    c = 0
    for i in range(0,N):
        for j in range(0,N):
            for k in range(0,N):
                pos[c][0] = (i+0.5)*D/N
                pos[c][1] = (j+0.5)*D/N
                pos[c][2] = (k+0.5)*D/N
                pos[c+1][0] = (i+1)*D/N
                pos[c+1][1] = (j+0.5)*D/N
                pos[c+1][2] = (k+0.5)*D/N
                pos[c+2][0] = (i+0.5)*D/N
                pos[c+2][1] = (j+1)*D/N
                pos[c+2][2] = (k+0.5)*D/N
                pos[c+3][0] = (i+0.5)*D/N
                pos[c+3][1] = (j+0.5)*D/N
                pos[c+3][2] = (k+1)*D/N
                c = c + 4
    return pos

def InitVel():
    vel = np.zeros((n,3))
    velmean = np.zeros((3)
    for c in range(0,n):
        for d in range(0,3):
            vel[c][d] = random.uniform(0,1)
            velmean[d] += vel[c][d]
    for c in range(0,n):
        for d in range(0,3):
            vel[c][d] -= velmean[d] / n
    return vel

SyntaxError: invalid syntax (<ipython-input-189-b300060534ba>, line 25)

In [190]:
#@jit(nopython=True)
def CalculateDistances(pos):
    CC = 0
    Dis = np.zeros((n**2))
    Cs = np.zeros((n**2))
    
    for i in range(0,n):
        for j in range(i+1,n):
            dx = pos[i][0] - pos[j][0]
            dy = pos[i][1] - pos[j][1]
            dz = pos[i][2] - pos[j][2]
            dx = dx - round(dx / (D)) * D
            dy = dy - round(dy / (D)) * D  
            dz = dz - round(dz / (D)) * D  
            d = math.sqrt(dx**2+dy**2+dz**2)
            Dis[i * n + j] = d
            if (d == 0):
                print("i = %g\t j = %g"%(i,j))
            if (d < MinDist):
                Cs[CC] = i*n+j
                CC += 1
    return point(Dis, Cs, CC)
                
@jit(nopython=True)
def CalculateForces(pos, CombC, Combs) :
    force = np.zeros((n,3))
    #force = [[0 for x in range(3)] for x in range(n)]
    for c in range(0,CombC):
        i = math.floor(Combs[c] / n)
        j = math.floor(Combs[c] % n)
        
        dx = pos[i][0] - pos[j][0]
        dy = pos[i][1] - pos[j][1]
        dz = pos[i][2] - pos[j][2]
        dx = dx - round(dx / (D)) * D
        dy = dy - round(dy / (D)) * D  
        dz = dz - round(dz / (D)) * D  
        d = math.sqrt(dx**2+dy**2+dz**2)
        dt = d**(-7)
        F = 12*(2*dt**2*d - dt)
        Fx = F * dx / d
        Fy = F * dy / d
        Fz = F * dz / d
        force[i][0] += Fx
        force[i][1] += Fy
        force[i][2] += Fz
        force[j][0] -= Fx
        force[j][1] -= Fy
        force[j][2] -= Fz
    return force

#@jit(nopython=True)
def CalulcatePotential(Distance):
    Pot = 0.0
    for i in range(0,n):
        for j in range(i+1,n):
            d = Distance[i*n+j]
            if (d == 0):
                print("i = %g\t j = %g"%(i,j))
            dt = d**(-6)
            Pot += 2*dt**2 - dt
    return Pot

pos = InitPos()
vel = InitVel()
    
ret = CalculateDistances(pos)
Combs = ret.Combs
CombC = ret.CombC
force = CalculateForces(pos, CombC, Combs)

In [197]:
#@jit
def Calculate():
    KinArr = []
    PotArr = []
    Time = []

    pos = InitPos()
    vel = InitVel()
    
    ret = CalculateDistances(pos)
    Combs = ret.Combs
    CombC = ret.CombC
    force = CalculateForces(pos, CombC, Combs)
    veltemp = vel
    t_last = 0
    t = 0
    while t <= t_end:
        if (t >= t_last):
            ret = CalculateDistances(pos)
            Distance = ret.Distance
            Combs = ret.Combs
            CombC = ret.CombC
            Pot = CalulcatePotential(Distance)
            # Calculate the Kinetic energy
            Kin = 0
            for c in range(0,n):
                Kin += math.sqrt(vel[c][0]**2+vel[c][1]**2+vel[c][2]**2)
            KinArr.append(Kin)
            PotArr.append(Pot)
            Time.append(t)
            #print("Time = %.3g"%(t))
            sys.stdout.write("\r%2.3g%%\t\t%g" % (math.floor(t/t_end * 1000)/10, CombC))
            sys.stdout.flush()
            t_last += 0.03
        
        vel += force*(dt/2)
        pos += vel*dt
        pos = pos%D
        force = CalculateForces(pos, CombC, Combs)
        vel += force*(dt/2)
        
        
        #for j in range(0,n):
         #   for i in range(0,3):
                #vel[j][i] += force[j][i] * dt/2
                #pos[j][i] += vel[j][i] * dt
                #pos[j][i] = pos[j][i]%D
        #force = CalculateForces(pos, CombC, Combs)
        #for j in range(0,n):
         #   for i in range(0,3):
              #vel[j][i] += force[j][i] * dt/2
        t += dt
        #print("Time = %.6f"%(time.time()-start_time))
    return point2(Time, KinArr, PotArr)
    
dt = 1E-3
ts = 1000
t_end = dt * ts
start_time = time.time()
ret = Calculate()
Time = ret.Time
KinArr = ret.Kinetic
PotArr = ret.Potential
time_spend = time.time() - start_time
time_it = time_spend/ts
print("\nTime = %.6f"%(time_it))

99%		985
Time = 0.024494


In [193]:
print("Starting")
#print("minDist = %.2g"%(MinDist))

t = 0
dt = 1E-3
t_end = dt * 1E2
#Calculate()
    
print("\nDone")

Starting

Done


In [192]:
plt.plot(Time, PotArr)
plt.plot(Time, KinArr)
#plt.plot(Time, PotArr + KinArr)
plt.legend(("Potential Energy", "Kinetic Energy", "Total Energy"))
plt.title("Energy in %g argo atoms"%(n))
plt.show()